In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

import os
import progressbar
from more_itertools import chunked

[WDM] - Downloading: 100%|██████████| 8.84M/8.84M [00:01<00:00, 7.38MB/s]


In [4]:
df = pd.read_csv('/Users/jang/Library/CloudStorage/GoogleDrive-gkfua00@gmail.com/내 드라이브/LiveCommerse/Channel_Rank_total.csv')
df

,Channel_Name,Channel_number,Num_Sub,Num_Video,Total_View,Best_View,Worst_View
0,산토리니,2,83.0,48,116.0,47.0,0.0
1,아디다스,3,21000.0,15,173516.0,100000.0,3426.0
2,구구구,4,36.0,38,138.0,27.0,0.0
3,뷰티테스트1_인디,5,2.0,18,218.0,47.0,2.0
4,리빙테스트_홈앤데코,6,0.0,0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
103803,솔직상점-,112196,24.0,0,0.0,0.0,0.0
103804,오얏트리,112197,5.0,0,0.0,0.0,0.0
103805,꽃피다 봄날,112198,13.0,0,0.0,0.0,0.0
103806,다롱세상,112199,10.0,0,0.0,0.0,0.0


In [5]:
temp_index = list(set(range(df.Channel_number.min(), df.Channel_number.max()+1)) - set(df.Channel_number))
temp_index.sort()

In [6]:
temp_index

[23,
 40,
 54,
 72,
 85,
 86,
 91,
 96,
 106,
 111,
 112,
 122,
 129,
 131,
 134,
 138,
 140,
 149,
 150,
 168,
 170,
 187,
 197,
 199,
 200,
 203,
 210,
 213,
 232,
 234,
 239,
 253,
 256,
 263,
 268,
 269,
 274,
 278,
 303,
 305,
 310,
 324,
 325,
 327,
 330,
 332,
 338,
 341,
 353,
 355,
 357,
 362,
 365,
 372,
 398,
 402,
 405,
 409,
 417,
 425,
 442,
 445,
 447,
 448,
 457,
 458,
 461,
 463,
 468,
 480,
 484,
 492,
 493,
 502,
 507,
 511,
 513,
 546,
 548,
 558,
 562,
 566,
 568,
 569,
 594,
 615,
 617,
 624,
 629,
 633,
 640,
 645,
 658,
 667,
 669,
 671,
 672,
 696,
 730,
 755,
 757,
 766,
 770,
 773,
 777,
 781,
 785,
 797,
 802,
 809,
 815,
 818,
 826,
 828,
 836,
 837,
 838,
 839,
 842,
 904,
 1142,
 1312,
 1652,
 1660,
 1934,
 2166,
 2285,
 2538,
 2613,
 2630,
 2674,
 2711,
 2765,
 2814,
 2872,
 2874,
 2887,
 3142,
 3588,
 3676,
 3733,
 3734,
 3739,
 3759,
 3837,
 3903,
 3999,
 4020,
 4165,
 4176,
 4227,
 4278,
 4355,
 4417,
 4499,
 4930,
 4998,
 5174,
 5238,
 5314,
 5885,
 

In [7]:
empty = []
df_livestar = pd.DataFrame({'Channel_Name' : empty, 'Channel_Number' : empty, 'Num_Sub' : empty, 'Num_Live' : empty})

bar = progressbar.ProgressBar(maxval=(len(temp_index))).start()
chrome_options = Options()
chrome_options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

for j, i in enumerate(temp_index):
  bar.update(j)

  url = f'https://shoppinglive.naver.com/channels/{i}'
  driver.get(url)
  time.sleep(1)
  soup = BeautifulSoup(driver.page_source, 'html.parser')

  Channel_name = str(soup.select('strong.ProfileLiveStar_name_eBbKy'))
  try:
    Channel_name = (Channel_name.split('<')[1])
    Channel_name = (Channel_name.split('>')[1])
    df_livestar.loc[i, ['Channel_Name']]= Channel_name
    df_livestar.loc[i, ['Channel_Number']]= int(i)
  except:
    continue
    
  try:

    Num_sub = str(soup.select('span.ProfileLiveStar_fan_KMCpf'))
    Num_sub = (Num_sub.split('<')[1])
    Num_sub = (Num_sub.split('>')[1])
    Num_sub = (Num_sub.split(' ')[1])
    Num_sub = (Num_sub.split('명')[0])

    if '만' in Num_sub:
        Num_sub = (Num_sub.split('만')[0])
        Num_sub = float(Num_sub) * 10000
        Num_sub = int(Num_sub)
    else:
        Num_sub = Num_sub.replace(',', '')
        Num_sub = int(Num_sub)

    df_livestar.loc[i, ['Num_Sub']]= Num_sub

  except:
    continue

  Num_Video = str(soup.select('div.ItemCount_wrap_dYzmP'))
  try:
    Num_Video = (Num_Video.split('<')[1])
    Num_Video = (Num_Video.split('>')[1])
    Num_Video = (Num_Video.split(' ')[1])
    df_livestar.loc[i, ['Num_Live']]= int(Num_Video)
    
  except:
    df_livestar.loc[i, ['Num_Live']]= 0

bar.finish()
df_livestar.to_csv('/Users/jang/Library/CloudStorage/GoogleDrive-gkfua00@gmail.com/내 드라이브/LiveCommerse/FUCKUGOOGLE/Live_star_rank.csv', index=False)

KeyboardInterrupt: 